<a href="https://colab.research.google.com/github/sandeep7348/Assignment/blob/master/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install wget


  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=2867c483db9f5f64955cdc120aa21ae4fc11e97e9892e3b14fa2c92fb877e5a0
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [4]:
!pip install gensim


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 53.6 MB/s eta 0:00:00


In [5]:
import gensim
from gensim.models import word2vec,keyedvectors


In [6]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2026-02-06 16:19:37--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.15.184.89, 16.15.223.68, 52.217.164.88, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.15.184.89|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2026-02-06 16:19:37 ERROR 404: Not Found.



In [7]:
import gensim.downloader as api
wv = api.load("word2vec-google-news-300")

print(wv["king"][:10])


[==================================================] 100.0% 1662.8/1662.8MB downloaded
[ 0.12597656  0.02978516  0.00860596  0.13964844 -0.02563477 -0.03613281
  0.11181641 -0.19824219  0.05126953  0.36328125]


In [8]:
wv.most_similar("woman")

[('man', 0.7664012908935547),
 ('girl', 0.7494640946388245),
 ('teenage_girl', 0.7336829304695129),
 ('teenager', 0.6317085027694702),
 ('lady', 0.6288785934448242),
 ('teenaged_girl', 0.6141784191131592),
 ('mother', 0.6076306104660034),
 ('policewoman', 0.6069462299346924),
 ('boy', 0.5975907444953918),
 ('Woman', 0.5770983099937439)]

In [16]:
vec=wv['king']-wv['man']+wv['woman']
print(wv.most_similar([vec]))
print(wv.similarity("cow","bottle"))


[('king', 0.8449392318725586), ('queen', 0.7300517559051514), ('monarch', 0.645466148853302), ('princess', 0.6156251430511475), ('crown_prince', 0.5818676352500916), ('prince', 0.5777117609977722), ('kings', 0.5613663792610168), ('sultan', 0.5376775860786438), ('Queen_Consort', 0.5344247817993164), ('queens', 0.5289887189865112)]
0.1480662


In [17]:
vec=wv['INR']-wv['India']+wv['GBP']
print(wv.most_similar([vec]))

[('GBP', 0.8160167336463928), ('INR', 0.7042931914329529), ('EUR', 0.6832454800605774), ('Pounds_Sterling', 0.6069923043251038), ('GBP###', 0.6049808859825134), ('#,###.##_PER_GBP', 0.5949068069458008), ('GBP#.##', 0.5900393724441528), ('GBP1', 0.5881087183952332), ('GBP##', 0.5874558687210083), ('GBP#.#m', 0.5831231474876404)]


In [26]:
print(wv.similarity('Man',"Speak"))
print(wv.similarity('rich',"Money"))

0.13370202
0.117453896


In [39]:
import nltk
nltk.download('punkt')      # tokenizer data
nltk.download('punkt_tab')  # sometimes needed in newer versions

import os
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess

story = []

for filename in os.listdir("sample_data"):
    if filename.endswith(".txt"):
        with open(os.path.join("sample_data", filename), "r", encoding="utf-8") as f:
            corpus = f.read()

        raw_sent = sent_tokenize(corpus)

        for sent in raw_sent:
            story.append(simple_preprocess(sent))

print(len(story))
model=gensim.models.Word2Vec(
    window=10,
    min_count=2,
  )
model.build_vocab(story)
model.train(story,total_examples=model.corpus_count,epochs=model.epochs)

print(model.wv.most_similar("man"))
print(model.wv.similarity("rich","money"))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


57136
[('bear', 0.7905452251434326), ('woman', 0.7559817433357239), ('nan', 0.7176592946052551), ('boy', 0.7173718810081482), ('knight', 0.6990187764167786), ('altogether', 0.6620523929595947), ('one', 0.6469284892082214), ('season', 0.6292943358421326), ('boasting', 0.6169681549072266), ('an', 0.6094112992286682)]
0.76794326


In [42]:
model.wv.doesnt_match(["queen","bran",'jon'])

'queen'

In [50]:
from sklearn.decomposition import PCA
pca=PCA(n_components=3)
X=pca.fit_transform(model.wv.get_normed_vectors())
print(X.shape)
import plotly.express as pt
fig=pt.scatter_3d(X[:100],x=0,y=1,z=2,color=model.wv.index_to_key[:100])
fig.show()

(11047, 3)
